**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install scikit-video

     |████████████████████████████████| 2.3MB 2.7MB/s 


In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten,Dropout

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The aim of the function act is to try to solve the exploration-exploitation dilemma. 
The environment does not reveal the reward of the actions not selected by the learner. Therefore, the learner should gain information by repeatedly selecting all actions. This is called exploration. However, each time the learner selects a bad action, it suffers some regret. The learner should then try and reduce the regret by repeatedly selecting the best action. This is called exploitation. The result is of these two opposite problems is the exploration-exploitation dilemma. One way of dealing with it is to use an $\epsilon$-greedy Algorithm which consist in exploring with probability $\epsilon$ by choosing a random action and exploiting with probability 1-$\epsilon$ by choosing the best action using the information already collected. If $\epsilon=1$, then we only explore and never try to reduce our regret. If $\epsilon=0$, then we never explore and have nearly no chance of finding the best policy. Therefore, $\epsilon$ is essential ($0<\epsilon<1$).


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=200 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The position array is used to represent the position of the rat on the grid at the current state. The board array is used to represent the current state of the game (cells still containing cheese or poison).


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        """Random Agent gives back a random action between the 4 actions possible"""
        return np.random.randint(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False
        
        win = 0
        lose = 0
 
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
 
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
 
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
 
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 14.0/12.0. Average score (2.0)
Win/lose count 12.5/16.0. Average score (-0.75)
Win/lose count 11.0/16.0. Average score (-2.1666666666666665)
Win/lose count 11.0/13.0. Average score (-2.125)
Win/lose count 10.5/9.0. Average score (-1.4)
Win/lose count 6.0/3.0. Average score (-0.6666666666666666)
Win/lose count 4.0/5.0. Average score (-0.7142857142857143)
Win/lose count 8.5/17.0. Average score (-1.6875)
Win/lose count 13.0/16.0. Average score (-1.8333333333333333)
Win/lose count 13.5/14.0. Average score (-1.7)
Win/lose count 10.0/9.0. Average score (-1.4545454545454546)
Win/lose count 8.0/10.0. Average score (-1.5)
Win/lose count 10.0/12.0. Average score (-1.5384615384615385)
Win/lose count 9.0/14.0. Average score (-1.7857142857142858)
Win/lose count 9.5/16.0. Average score (-2.1)
Win/lose count 15.0/13.0. Average score (-1.84375)
Win/lose count 11.0/10.0. Average score (-1.6764705882352942)
Win/lose count 9.0/23.0. Average score (-2.361111111111111)
Win/lose count 16.0/16

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




Using the expression of the $Q$-function given in the introduction, we have:

\begin{equation*}\begin{split}Q^\pi(s,a)&=E_{p^{\pi}}[\sum_{t=0}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
&=r(s,a)+E_{p^{\pi}}[\sum_{t=1}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
&=r(s,a)+\gamma \sum_{s',a'} p(s',a'|s,a) E_{p^{\pi}}[\sum_{t=1}^{+\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{1}=s',a_{1}=a'] \> (Markov \> Property) \\
&=r(s,a)+\gamma \sum_{s',a'} p(s',a'|s,a) E_{p^{\pi}}[\sum_{t=0}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a'] \> (change \> of \> time) \\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{split}\end{equation*}




Using the definition of the optimal Q function, we have:

\begin{equation*}\begin{split}
Q^*(s,a)&=\max_{\pi} \> Q^\pi(s,a) \\
&=\max_{\pi} \> r(s,a)+\gamma \sum_{s'} p(s'|s,a) \sum_{a'} \pi(a'|s')Q^{\pi}(s',a') \\
&=r(s,a)+\gamma \max_{\pi} \> \sum_{s'} p(s'|s,a) \sum_{a'} \pi(a'|s')Q^{\pi}(s',a')
\end{split}\end{equation*}

As $\forall s', \sum_{a'} \pi(a'|s')=1$ and $\forall (s',a'), \pi(a'|s')\geq0$ then we have that $\forall s', \sum_{a'} \pi(a'|s')Q^{\pi}(s',a')\leq (\sum_{a''} \pi(a''|s'))\max_{a'} Q^{\pi}(s',a')=\max_{a'} Q^{\pi}(s',a')$. 

Moreover, each policy $\pi=(a',\pi')$ so we have $$\max_{(a',\pi')} \> \sum_{s'} p(s'|s,a) \sum_{a'} \pi(a'|s')Q^{\pi}(s',a')\leq \sum_{s'} p(s'|s,a) \max_{a'} (\max_{\pi'} Q^{\pi'}(s',a'))=\sum_{s'} p(s'|s,a) \max_{a'}Q^*(s',a')$$

Lets take $\hat{\pi}$ such that $\forall s', \hat{\pi}(s')\in argmax_{a} Q^*(s,a)$: 
$$\sum_{s'} p(s'|s,a) \max_{a'}Q^*(s',a')=\sum_{s'} p(s'|s,a)Q^{\hat{\pi}}(s',\hat{\pi}(s'))\leq \max_{(a',\pi')} \> \sum_{s'} p(s'|s,a) \sum_{a'} \pi(a'|s')Q^{\pi}(s',a')$$

Therefore, 
\begin{equation*}\begin{split}
Q^*(s,a)&=\max_{\pi} \> r(s,a)+\gamma \sum_{s'} p(s'|s,a) \sum_{a'} \pi(a'|s')Q^{\pi}(s',a') \\
&=r(s,a)+\gamma\sum_{s'} p(s'|s,a) \max_{a'} Q^*(s',a') \\
&=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{split}\end{equation*}

We can easily see that we could use as objective function:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

Indeed, the minimum of this function is obtained for $\mathcal{L}(\theta^*)=0$ which implies that $Q(s,a,\theta)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]=Q^*(s,a)$.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)<self.max_memory:
          self.memory.append(m)
        else:
          self.memory=self.memory[:-1]+[m]

    def random_access(self):
        n=np.random.randint(len(self.memory))
        return self.memory[n]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

        #Target network updates
        self.count=0

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.array([s]))[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_=self.memory.random_access()
            input_states[i]=s_
            target_q[i]=self.model.predict(np.array([s_]))
            if game_over_:
              target_q[i][a_]=r_
            else:
              target_q[i][a_]=r_+self.discount*np.max(self.model.predict(np.array([n_s_])))
        self.count+=1
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -2.5, 1)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(16))
        model.add(Activation("relu"))
        model.add(Dense(16))
        model.add(Activation("relu"))
        model.add(Dense(4))
        

        model.compile(keras.optimizers.adam(lr=lr), "mse")
        self.model = model


In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.0001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/200 | Loss 0.0081 | Win/lose count 5.0/3.0 (2.0)
Epoch 001/200 | Loss 0.0080 | Win/lose count 3.5/5.0 (-1.5)
Epoch 002/200 | Loss 0.0201 | Win/lose count 2.0/1.0 (1.0)
Epoch 003/200 | Loss 0.0056 | Win/lose count 2.5/4.0 (-1.5)
Epoch 004/200 | Loss 0.0044 | Win/lose count 4.0/3.0 (1.0)
Epoch 005/200 | Loss 0.0150 | Win/lose count 6.5/7.0 (-0.5)
Epoch 006/200 | Loss 0.0146 | Win/lose count 6.5/5.0 (1.5)
Epoch 007/200 | Loss 0.0101 | Win/lose count 7.0/4.0 (3.0)
Epoch 008/200 | Loss 0.0120 | Win/lose count 5.0/8.0 (-3.0)
Epoch 009/200 | Loss 0.0060 | Win/lose count 4.0/5.0 (-1.0)
Epoch 010/200 | Loss 0.0100 | Win/lose count 6.0/1.0 (5.0)
Epoch 011/200 | Loss 0.0097 | Win/lose count 6.5/8.0 (-1.5)
Epoch 012/200 | Loss 0.0102 | Win/lose count 3.0/2.0 (1.0)
Epoch 013/200 | Loss 0.0125 | Win/lose count 4.5/4.0 (0.5)
Epoch 014/200 | Loss 0.0049 | Win/lose count 1.5/0 (1.5)
Epoch 015/200 | Loss 0.0165 | Win/lose count 2.5/3.0 (-0.5)
Epoch 016/200 | Loss 0.0025 | Win/lose count 6.0/2.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(64, (3, 3), padding="same", activation="relu",input_shape=(5,5,self.n_state)))
        model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
        model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(keras.optimizers.adam(lr=lr), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.0001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/200 | Loss 0.0005 | Win/lose count 4.5/4.0 (0.5)
Epoch 001/200 | Loss 0.0041 | Win/lose count 4.0/6.0 (-2.0)
Epoch 002/200 | Loss 0.0017 | Win/lose count 7.5/5.0 (2.5)
Epoch 003/200 | Loss 0.0012 | Win/lose count 4.5/7.0 (-2.5)
Epoch 004/200 | Loss 0.0003 | Win/lose count 5.0/4.0 (1.0)
Epoch 005/200 | Loss 0.0005 | Win/lose count 8.5/2.0 (6.5)
Epoch 006/200 | Loss 0.0005 | Win/lose count 8.5/1.0 (7.5)
Epoch 007/200 | Loss 0.0049 | Win/lose count 7.5/1.0 (6.5)
Epoch 008/200 | Loss 0.0002 | Win/lose count 4.5/3.0 (1.5)
Epoch 009/200 | Loss 0.0058 | Win/lose count 3.0/2.0 (1.0)
Epoch 010/200 | Loss 0.0014 | Win/lose count 6.5/3.0 (3.5)
Epoch 011/200 | Loss 0.0002 | Win/lose count 3.5/2.0 (1.5)
Epoch 012/200 | Loss 0.0071 | Win/lose count 1.0/2.0 (-1.0)
Epoch 013/200 | Loss 0.0002 | Win/lose count 6.5/2.0 (4.5)
Epoch 014/200 | Loss 0.0003 | Win/lose count 6.5/2.0 (4.5)
Epoch 015/200 | Loss 0.0019 | Win/lose count 8.0/5.0 (3.0)
Epoch 016/200 | Loss 0.0018 | Win/lose count 3.0/0 (3

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 9.5/1.0. Average score (8.5)
Win/lose count 9.0/2.0. Average score (7.75)
Win/lose count 9.0/7.0. Average score (5.833333333333333)
Win/lose count 9.0/0. Average score (6.625)
Win/lose count 13.5/2.0. Average score (7.6)
Win/lose count 4.0/4.0. Average score (6.333333333333333)
Win/lose count 6.0/6.0. Average score (5.428571428571429)
Win/lose count 3.0/2.0. Average score (4.875)
Win/lose count 5.5/3.0. Average score (4.611111111111111)
Win/lose count 11.5/6.0. Average score (4.7)
Win/lose count 5.5/2.0. Average score (4.590909090909091)
Win/lose count 3.5/2.0. Average score (4.333333333333333)
Win/lose count 10.5/3.0. Average score (4.576923076923077)
Win/lose count 10.0/2.0. Average score (4.821428571428571)
Win/lose count 19.0/2.0. Average score (5.633333333333334)
Win/lose count 12.0/3.0. Average score (5.84375)
Win/lose count 5.5/1.0. Average score (5.764705882352941)
Win/lose count 8.5/2.0. Average score (5.805555555555555)
Win/lose count 6.5/3.0. A

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

One of the main issues is that, for both algorithms, the rat quickly tends to turn in circles and so a large part of the map stays unexplored. Indeed, in a few set of states, nothing seems to attract the rat and it keeps going back and forward. This is why the rat gets easily stuck for lower temperatures. Furthermore, we also notice that the performance of our algorithm is better if we keep exploring with a probability $\epsilon$ during the test (agent.act(state,train=True)) which seems logical according to the issue we've just stated.We can also note that both algorithms get far better results than the RandomAgent which shows that our algorithms are a big improvement. Moreover, CNN seems to outperform FC and we will therefore keep this architecture for the next question.



***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    #Linear decreasing in epsilon to 0.1
    eps_init=agent.epsilon
    if eps_init<=0.1:
      alpha=0
    if eps_init>0.1:
      if epoch>200:
        alpha=(eps_init-0.1)/20
      else:
        alpha=(eps_init-0.1)/(max(1,epoch//10))

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        if e%10==0 and e<=200:
          agent.set_epsilon(eps_init-(e//10)*alpha)
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action,train=True)
            
            #Change reward so we can compare the new algorithm with the old ones
            reward_compare=reward+env.last_malus_position
            # Update the counters
            if reward_compare > 0:
                win = round(win + reward_compare,2)
            if reward_compare < 0:
                lose = round(lose - reward_compare,2)

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size=grid_size, max_time=max_time, temperature=temperature)

        #New state: visited cells
        self.malus_position=np.zeros((self.grid_size, self.grid_size))
        self.last_malus_position=0
    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        #visited state
        reward = 0
        if train:
          reward = -self.malus_position[self.x, self.y]
          self.last_malus_position=self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.2
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        #Visited cells
        self.malus_position=np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x,self.y] = 0.2
        self.last_malus_position=0
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    

In [21]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.0001, epsilon = 0.9, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 300, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/300 | Loss 0.0434 | Win/lose count 9.5/18.0 (-8.5)
Epoch 001/300 | Loss 0.0071 | Win/lose count 11.0/18.0 (-7.0)
Epoch 002/300 | Loss 0.0071 | Win/lose count 16.5/12.0 (4.5)
Epoch 003/300 | Loss 0.0028 | Win/lose count 15.0/15.0 (0.0)
Epoch 004/300 | Loss 0.0058 | Win/lose count 8.0/12.0 (-4.0)
Epoch 005/300 | Loss 0.0027 | Win/lose count 11.5/19.0 (-7.5)
Epoch 006/300 | Loss 0.0044 | Win/lose count 9.5/15.0 (-5.5)
Epoch 007/300 | Loss 0.0022 | Win/lose count 15.5/17.0 (-1.5)
Epoch 008/300 | Loss 0.0034 | Win/lose count 15.5/19.0 (-3.5)
Epoch 009/300 | Loss 0.0043 | Win/lose count 11.0/17.0 (-6.0)
Epoch 010/300 | Loss 0.0046 | Win/lose count 14.5/19.0 (-4.5)
Epoch 011/300 | Loss 0.0022 | Win/lose count 11.0/9.0 (2.0)
Epoch 012/300 | Loss 0.0027 | Win/lose count 12.0/19.0 (-7.0)
Epoch 013/300 | Loss 0.0019 | Win/lose count 9.0/12.0 (-3.0)
Epoch 014/300 | Loss 0.0027 | Win/lose count 14.0/24.0 (-10.0)
Epoch 015/300 | Loss 0.0016 | Win/lose count 11.5/16.0 (-4.5)
Epoch 016/300 |

In [22]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 16.5/6.0. Average score (10.5)
Win/lose count 13.5/4.0. Average score (10.0)
Win/lose count 23.5/6.0. Average score (12.5)
Win/lose count 23.0/6.0. Average score (13.625)
Win/lose count 26.0/6.0. Average score (14.9)
Win/lose count 20.5/3.0. Average score (15.333333333333334)
Win/lose count 17.0/6.0. Average score (14.714285714285714)
Win/lose count 15.0/4.0. Average score (14.25)
Win/lose count 24.5/4.0. Average score (14.944444444444445)
Win/lose count 22.5/7.0. Average score (15.0)
Win/lose count 8.0/2.0. Average score (14.181818181818182)
Win/lose count 19.5/6.0. Average score (14.125)
Win/lose count 14.5/1.0. Average score (14.076923076923077)
Win/lose count 20.0/6.0. Average score (14.071428571428571)
Win/lose count 22.0/8.0. Average score (14.066666666666666)
Win/lose count 20.5/5.0. Average score (14.15625)
Win/lose count 25.0/3.0. Average score (14.617647058823529)
Win/lose count 31.5/0. Average score (15.555555555555555)
Win/lose count 15.0/2.0. Average score (

We can see through the video and the results that the rat doesn't get stucked anymore and, therefore, is able to eat more cheese. The CNN with an environment which encourages exploration outperforms all the precedent models. 

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***